In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql.functions import col, udf
import requests

# Azure OpenAI Config
AZURE_OPENAI_ENDPOINT = "https://mehrdad-ai-sentiment-01.openai.azure.com/"
AZURE_OPENAI_API_KEY = "7hPkYyHguA6O8y55zJW8J2chAQgDF0JVL8DTbDAejDOmydgukDQLJQQJ99BBACYeBjFXJ3w3AAABACOG9quD"
DEPLOYMENT_NAME = "gpt-4o"

headers = {
    "Content-Type": "application/json",
    "api-key": AZURE_OPENAI_API_KEY
}

StatementMeta(, 0991c303-da87-4872-b9d1-d7a99ffb8678, 3, Finished, Available, Finished)

In [2]:
def analyze_sentiment(text):
    try:
        payload = {
            "messages": [{"role": "system", "content": "Analyze sentiment as Positive, Neutral, or Negative"},
                         {"role": "user", "content": text}],
            "max_tokens": 100
        }

        response = requests.post(
            f"{AZURE_OPENAI_ENDPOINT}openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version=2023-07-01-preview",
            headers=headers, json=payload
        )

        sentiment = response.json()["choices"][0]["message"]["content"]
        return sentiment
    except Exception as e:
        return "Error"

analyze_sentiment_udf = udf(analyze_sentiment)

StatementMeta(, 0991c303-da87-4872-b9d1-d7a99ffb8678, 4, Finished, Available, Finished)

In [3]:
df = spark.read.format("delta").load("Tables/raw_customer_feedback")
df.show()

StatementMeta(, 0991c303-da87-4872-b9d1-d7a99ffb8678, 5, Finished, Available, Finished)

+-----------+--------------------+
|Customer_ID|            Feedback|
+-----------+--------------------+
|        101|Horrible experien...|
|        102|Customer service ...|
|        103|The service was e...|
|        104|Would definitely ...|
|        105|Would definitely ...|
|        106|Didn't meet my ex...|
|        107|Great ambiance an...|
|        108|The product quali...|
|        109|Food was cold and...|
|        110|Great ambiance an...|
|        111|Great ambiance an...|
|        112|Waited too long f...|
|        113|They went above a...|
|        114|Customer service ...|
|        115|Loved the friendl...|
|        116|The service was e...|
|        117|Very disappointed...|
|        118|Fantastic place, ...|
|        119|It was okay, noth...|
|        120|Customer service ...|
+-----------+--------------------+
only showing top 20 rows



In [4]:
df_enriched = df.withColumn("Sentiment", analyze_sentiment_udf(col("Feedback")))
df_enriched.write.format("delta").mode("overwrite").save("Tables/customer_feedback_enriched")
df_enriched.show()

StatementMeta(, 0991c303-da87-4872-b9d1-d7a99ffb8678, 6, Finished, Available, Finished)

+-----------+--------------------+--------------------+
|Customer_ID|            Feedback|           Sentiment|
+-----------+--------------------+--------------------+
|        101|Horrible experien...|            Negative|
|        102|Customer service ...|            Negative|
|        103|The service was e...|            Positive|
|        104|Would definitely ...|            Positive|
|        105|Would definitely ...|            Positive|
|        106|Didn't meet my ex...|            Negative|
|        107|Great ambiance an...|            Positive|
|        108|The product quali...|**Sentiment: Posi...|
|        109|Food was cold and...|            Negative|
|        110|Great ambiance an...|            Positive|
|        111|Great ambiance an...|            Positive|
|        112|Waited too long f...|            Negative|
|        113|They went above a...|            Positive|
|        114|Customer service ...|            Negative|
|        115|Loved the friendl...|            Po